In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_test_data = pd.concat([train_data, test_data])

In [4]:
train_test_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0.0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0.0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
train_test_data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [6]:
train_test_data.shape

(1309, 12)

In [6]:
train_test_data['Cabin'] = train_test_data['Cabin'].str[:1]

In [7]:
object_columns = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

for column in object_columns:
    train_test_data[column] = pd.factorize(train_test_data[column])[0]

In [8]:
train_test_data['Cabin'] = train_test_data['Cabin'].replace(-1, np.nan)

In [9]:
train_test_data['Cabin'].fillna(train_test_data.groupby('Pclass')['Cabin'].transform('median'), inplace=True)

In [10]:
train_test_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,0,0,22.0,1,0,0,7.2500,4.0,0
1,2,1.0,1,1,1,38.0,1,0,1,71.2833,0.0,1
2,3,1.0,3,2,1,26.0,0,0,2,7.9250,4.0,0
3,4,1.0,1,3,1,35.0,1,0,3,53.1000,0.0,0
4,5,0.0,3,4,0,35.0,0,0,4,8.0500,4.0,0
5,6,0.0,3,5,0,NaN,0,0,5,8.4583,4.0,2
6,7,0.0,1,6,0,54.0,0,0,6,51.8625,1.0,0
7,8,0.0,3,7,0,2.0,3,1,7,21.0750,4.0,0
8,9,1.0,3,8,1,27.0,0,2,8,11.1333,4.0,0
9,10,1.0,2,9,1,14.0,1,0,9,30.0708,6.0,1


In [11]:
train_data = train_test_data[~train_test_data['Survived'].isnull()]
test_data = train_test_data[train_test_data['Survived'].isnull()]

In [12]:
ftr_train = train_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1)
target = train_data['Survived']

In [13]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_train, target, test_size=0.3, random_state=2022)

In [14]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

bayesian_params = {
    'max_depth': (2, 8), 
    'num_leaves': (2, 12), 
    'min_child_samples': (1, 50), 
    'min_child_weight':(1, 20),
    'subsample':(0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

In [15]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":100, "learning_rate":0.02,
        'max_depth': int(round(max_depth)),
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 50, 
                early_stopping_rounds= 50)
    valid_proba = lgb_model.predict_proba(valid_x)[:, 1]
    roc_auc = roc_auc_score(valid_y, valid_proba)
    
    return roc_auc   

In [16]:
lgbB0 = BayesianOptimization(f=lgb_roc_eval, pbounds=bayesian_params, random_state=2022)
lgbB0.maximize(init_points=5, n_iter=25)

|   iter    |  target   |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
[50]	training's auc: 0.845481	training's binary_logloss: 0.551161	valid_1's auc: 0.851845	valid_1's binary_logloss: 0.551627
|  1        |  0.8518   |  14.59    |  4.994    |  6.556    |  1.95     |  8.854    |  24.35    |  8.977    |  0.8237   |
[50]	training's auc: 0.820232	training's binary_logloss: 0.603963	valid_1's auc: 0.852083	valid_1's binary_logloss: 0.595709
|  2        |  0.8521   |  449.5    |  6.327    |  41.74    |  16.72    |  10.34    |  47.85    |  3.681    |  0.7474   |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[50]	training's auc: 0.853615	training's binary_logloss: 0.514307	valid_1's auc: 0.83753	valid_1's binary_logloss: 0.524435
[100]	training's auc: 0.857324	training's binary_logloss: 0.471569	valid_1's auc: 0.84631	valid_1's binary_logloss: 0.483002
|  3        |  0.8463   |  176.4    |  5.717    |  48.9     |  2.832    |  9.442    |  14.63    |  2.987    |  0.8762   |
[50]	training's auc: 0.842129	training's binary_logloss: 0.546798	valid_1's auc: 0.85244	valid_1's binary_logloss: 0.547076


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[100]	training's auc: 0.842802	training's binary_logloss: 0.509751	valid_1's auc: 0.853036	valid_1's binary_logloss: 0.509269
|  4        |  0.8527   |  19.15    |  5.142    |  43.36    |  8.388    |  4.122    |  23.76    |  5.647    |  0.6747   |
[50]	training's auc: 0.826181	training's binary_logloss: 0.596646	valid_1's auc: 0.852321	valid_1's binary_logloss: 0.589651
|  5        |  0.8523   |  488.2    |  2.227    |  39.92    |  7.8      |  9.48     |  45.73    |  3.727    |  0.9824   |


...

/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[50]	training's auc: 0.828626	training's binary_logloss: 0.565193	valid_1's auc: 0.850595	valid_1's binary_logloss: 0.562887
|  26       |  0.8506   |  490.1    |  2.845    |  38.36    |  17.3     |  10.23    |  31.7     |  6.296    |  0.5999   |
[50]	training's auc: 0.826181	training's binary_logloss: 0.588638	valid_1's auc: 0.852321	valid_1's binary_logloss: 0.582569
|  27       |  0.8523   |  485.5    |  2.923    |  7.683    |  3.172    |  8.357    |  43.76    |  1.026    |  0.9643   |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[50]	training's auc: 0.828626	training's binary_logloss: 0.555808	valid_1's auc: 0.850595	valid_1's binary_logloss: 0.554712
|  28       |  0.8506   |  494.9    |  7.71     |  2.615    |  15.76    |  10.97    |  29.85    |  2.252    |  0.6614   |
[50]	training's auc: 0.820232	training's binary_logloss: 0.611024	valid_1's auc: 0.852083	valid_1's binary_logloss: 0.601957
|  29       |  0.8521   |  498.4    |  4.122    |  3.151    |  18.51    |  9.075    |  49.76    |  5.456    |  0.6832   |
[50]	training's auc: 0.827943	training's binary_logloss: 0.59988	valid_1's auc: 0.850417	valid_1's binary_logloss: 0.592773
|  30       |  0.8506   |  481.8    |  4.931    |  1.737    |  19.05    |  9.412    |  45.86    |  6.801    |  0.7783   |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


In [17]:
lgbB0.res

[{'target': 0.8518452380952382,
  'params': {'max_bin': 14.585720765804705,
   'max_depth': 4.994346865111928,
   'min_child_samples': 6.555800806806923,
   'min_child_weight': 1.949506345374979,
   'num_leaves': 8.854075942430917,
   'reg_alpha': 24.354533530104362,
   'reg_lambda': 8.976674607206252,
   'subsample': 0.8237260353716331}},
 {'target': 0.8520833333333333,
  'params': {'max_bin': 449.5119301675884,
   'max_depth': 6.326809574988113,
   'min_child_samples': 41.7363176180333,
   'min_child_weight': 16.72379330761429,
   'num_leaves': 10.335795840581405,
   'reg_alpha': 47.85264633978365,
   'reg_lambda': 3.681076325612883,
   'subsample': 0.7474188149426438}},
 {'target': 0.8463095238095237,
  'params': {'max_bin': 176.3596425478705,
   'max_depth': 5.716575957469941,
   'min_child_samples': 48.89895228264298,
   'min_child_weight': 2.8322284751214624,
   'num_leaves': 9.442062122546155,
   'reg_alpha': 14.632048710028132,
   'reg_lambda': 2.9874548342985574,
   'subsample

In [18]:
target_list = []
for result in lgbB0.res:
    target = result['target']
    target_list.append(target)

print(target_list)
print('maximum target index', np.argmax(np.array(target_list)))

[0.8518452380952382, 0.8520833333333333, 0.8463095238095237, 0.8527083333333333, 0.8523214285714287, 0.8474702380952381, 0.8513988095238096, 0.848154761904762, 0.8505952380952381, 0.780595238095238, 0.8469345238095238, 0.8520833333333333, 0.837232142857143, 0.8508630952380953, 0.8520833333333333, 0.780595238095238, 0.8520833333333333, 0.8508630952380953, 0.8520833333333333, 0.8520833333333333, 0.8520833333333333, 0.8520833333333333, 0.837232142857143, 0.8520833333333333, 0.8508630952380953, 0.8505952380952381, 0.8523214285714287, 0.8505952380952381, 0.8520833333333333, 0.8505952380952381]
maximum target index 3


In [19]:
max_dict = lgbB0.res[np.argmax(np.array(target_list))]
print(max_dict)

{'target': 0.8527083333333333, 'params': {'max_bin': 19.145226575486895, 'max_depth': 5.142424615909564, 'min_child_samples': 43.35735651281808, 'min_child_weight': 8.388013965015574, 'num_leaves': 4.1219184946689085, 'reg_alpha': 23.764283405999475, 'reg_lambda': 5.647159507234463, 'subsample': 0.6747146479983777}}


In [20]:
clf = LGBMClassifier(
    n_jobs=-1,
    n_estimators=50,
    learning_rate=0.02,
    max_bin=19,
    min_child_samples=43,
    min_child_weight=8,
    num_leaves=4,
    subsample=0.982,
    max_depth=5,
    reg_alpha=23.764,
    reg_lambda=5.647,
    silent=-1,
    verbose=-1
)

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=30,
       early_stopping_rounds=30)

[30]	training's auc: 0.842433	training's binary_logloss: 0.577363	valid_1's auc: 0.850357	valid_1's binary_logloss: 0.578088


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verb

LGBMClassifier(learning_rate=0.02, max_bin=19, max_depth=5,
               min_child_samples=43, min_child_weight=8, n_estimators=50,
               num_leaves=4, reg_alpha=23.764, reg_lambda=5.647, silent=-1,
               subsample=0.982, verbose=-1)

In [21]:
preds = clf.predict_proba(test_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1))[:, 1]

In [22]:
preds.tolist()

[0.2937638794965778,
 0.4007266744970496,
 0.2937638794965778,
 0.2937638794965778,
 0.4007266744970496,
 0.2937638794965778,
 0.4007266744970496,
 0.3622788280846253,
 0.4007266744970496,
 0.2937638794965778,
 0.2937638794965778,
 0.3174429555325261,
 0.5883201932799521,
 0.2937638794965778,
 0.5883201932799521,
 0.5883201932799521,
 0.2937638794965778,
 0.2937638794965778,
 0.4007266744970496,
 0.4007266744970496,
 0.3745505990066086,
 0.30612646073154004,
 0.5883201932799521,
 0.3884430176565008,
 0.5883201932799521,
 0.2937638794965778,
 0.5883201932799521,
 0.2937638794965778,
 0.3745505990066086,
 0.2937638794965778,
 0.2937638794965778,
 0.3487884725369975,
 0.4007266744970496,
 0.4007266744970496,
 0.3745505990066086,
 0.2937638794965778,
 0.4007266744970496,
 0.4007266744970496,
 0.2937638794965778,
 0.35031445706076497,
 0.30612646073154004,
 0.3745505990066086,
 0.2937638794965778,
 0.5883201932799521,
 0.5883201932799521,
 0.2937638794965778,
 0.3745505990066086,
 0.2937638

In [23]:
preds = [1 if x > 0.5 else 0 for x in preds]

In [24]:
test_data['Survived'] = preds

/var/folders/09/xdnxbw6n7bl58mxczkp01qq40000gn/T/ipykernel_30718/1324259112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Survived'] = preds


In [25]:
test_data[['PassengerId', 'Survived']].to_csv('titanic_hyperparameter_tuning_2.csv', index=False)